# Import

In [ ]:
# 라이브러리 설치
!pip install --upgrade pip
!pip install pandas scikit-learn xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1


In [ ]:
# macOS에서 xgboost를 사용하기 위한 libomp 설치 명령어
!brew install libomp

To reinstall 20.1.7, run:
  brew reinstall libomp


In [ ]:
# 설치된 libomp의 경로 확인
!brew --prefix libomp

/opt/homebrew/opt/libomp


In [ ]:
# .zshrc에 DYLD_LIBRARY_PATH를 설정
libomp_path = "/opt/homebrew/opt/libomp/lib"

username=""
with open(f'/Users/{username}/.zshrc', 'a') as f:
    f.write(f'\nexport DYLD_LIBRARY_PATH="{libomp_path}:$DYLD_LIBRARY_PATH"\n')

In [ ]:
# 환경변수 설정 적용
!source ~/.zshrc

                    'c.
                 ,xNMM.
               .OMMMMo
               OMMM0,
     .;loddo:' loolloddol;.
   cKMMMMMMMMMMNWMMMMMMMMMM0:
 .KMMMMMMMMMMMMMMMMMMMMMMMWd.
 XMMMMMMMMMMMMMMMMMMMMMMMX.
;MMMMMMMMMMMMMMMMMMMMMMMM:
:MMMMMMMMMMMMMMMMMMMMMMMM:
.MMMMMMMMMMMMMMMMMMMMMMMMX.
 kMMMMMMMMMMMMMMMMMMMMMMMMWd.
 .XMMMMMMMMMMMMMMMMMMMMMMMMMMk
  .XMMMMMMMMMMMMMMMMMMMMMMMMK.
    kMMMMMMMMMMMMMMMMMMMMMMd
     ;KMMMMMMMWXXWMMMMMMMk.
       .cooc,.    .,coo:.
kimmandoo@KIMMANDOOui-MacBookPro.local 
-------------------------------------- 
OS: macOS 15.5 24F74 arm64 
Host: MacBookPro18,1 
Kernel: 24.5.0 
Uptime: 8 days, 18 hours, 58 mins 
Packages: 3 (port), 94 (brew) 
Shell: zsh 5.9 
Resolution: 1728x1117, 1080x810 
DE: Aqua 
WM: Spectacle 
Terminal: python 
CPU: Apple M1 Pro 
GPU: Apple M1 Pro 
Memory: 3126MiB / 16384MiB 

                        
                        




In [ ]:
# 라이브러리 불러오기
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier # 베이스라인 코드에서는 XGBoost 모델로 분류

# Data Load & Split

In [ ]:
# train, test 데이터 불러오기
train = pd.read_csv('./train.csv', encoding='utf-8-sig')
test = pd.read_csv('./test.csv', encoding='utf-8-sig')

In [ ]:
# 모델 학습을 위한 데이터 분리
X = train[['title', 'full_text']] # 모델 학습 입력 컬럼 (title, full_text)
y = train['generated']            # 정답 라벨 컬럼 (generated / AI: 1, 사람: 0)
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# TF-IDF Vectorization

In [ ]:
# TF-IDF 벡터화
get_title = FunctionTransformer(lambda x: x['title'], validate=False)      # title 컬럼만 추출하는 용도
get_text = FunctionTransformer(lambda x: x['full_text'], validate=False)   # full_text 컬럼만 추출하는 용도

# TF-IDF 벡터라이저 구성
# title, full_text 컬럼 별도의 TF-IDF 벡터라이저 구성 후 병합
vectorizer = FeatureUnion([
    ('title', Pipeline([('selector', get_title),
                        ('tfidf', TfidfVectorizer(ngram_range=(1,2), max_features=3000))])),
    ('full_text', Pipeline([('selector', get_text), 
                            ('tfidf', TfidfVectorizer(ngram_range=(1,2), max_features=10000))])),
]) 

# 피처 변환
X_train_vec = vectorizer.fit_transform(X_train) # X_train(title, full_text)에 대해 벡터라이저 학습 후 벡터로 변환
X_val_vec = vectorizer.transform(X_val)         # X_val(title, full_text)에 대해 벡터라이저 학습 후 벡터로 변환

# Train

In [ ]:
# 모델 정의
xgb = XGBClassifier(random_state=42)            # XGBoost 기반 이진 분류기 정의
xgb.fit(X_train_vec, y_train)                   # 모델 학습

val_probs = xgb.predict_proba(X_val_vec)[:, 1]  # 검증 데이터에 대한 클래스별 확률 예측 (AI로 판단될 확률만 가져옴)
auc = roc_auc_score(y_val, val_probs)           # AUC 계산 (1에 가까울수록 좋음, 0.5면 랜덤 수준)
print(f"Validation AUC: {auc:.4f}")             # AUC 소수점 넷째자리까지 출력

Validation AUC: 0.9113


# Inference

In [ ]:
# test용으로 'paragraph_text'를 'full_text'에 맞게 재명명
test = test.rename(columns={'paragraph_text': 'full_text'}) # train과 같은 구조여야 같은 벡터라이저 통과 가능
X_test = test[['title', 'full_text']]                       # 모델 입력으로 사용될 컬럼만 추출, train과 동일한 형태

X_test_vec = vectorizer.transform(X_test)                   # 학습 때 사용한 벡터라이저를 사용해 X_test를 벡터로 변환

probs = xgb.predict_proba(X_test_vec)[:, 1]                 # 테스트 데이터에 대해 클래스별 확률 예측 (AI로 판단될 확률만 가져옴)

# Submission

In [ ]:
# 결과 저장하기
sample_submission = pd.read_csv('./sample_submission.csv', encoding='utf-8-sig') # 기존에 제공된 제출 양식 파일 불러오기
sample_submission['generated'] = probs                                           # generated 컬럼에 모델이 예측한 AI일 확률값(probs)을 채움

sample_submission.to_csv(f'./baseline_submission.csv', index=False)              # 최종 제출 파일로 저장